In [ ]:
include("setup.jl")

# Facility location problem

**Originally Contributed by**: Mathieu Tanneau and Alexis Montoison

Benchmark instances: http://resources.mpi-inf.mpg.de/departments/d1/projects/benchmarks/UflLib/

## Uncapacitated facility location

### Problem description

* $M=\{1, \dots, m\}$ clients, $N=\{ 1, \dots, n\}$ sites where a facility can be built.

In [ ]:
n = 2  # number of facility locations
m = 5  # number of clients

* $f_j$: fixed cost of building a facility at site $j$
* $c_{i, j}$: cost for serving customer $i$ from facility $j$

In [ ]:
# Draw costs
Random.seed!(0)
f = rand(1000:10000, n);
c = rand(0:1000, m, n);

### MILP formulation

$$
    \begin{array}{cl}
        \min_{x, y} \ \ \ &
            \sum_{i, j} c_{i, j} x_{i, j} + 
            \sum_{j} f_{j} y_{j}\\
        s.t. &
            \sum_{j} x_{i, j} = 1, \ \ \ \forall i \in M\\
        &   x_{i, j} \leq y_{j}, \ \ \ \forall i \in M, j \in N\\
        &   x_{i, j}, y_{j} \in \{0, 1\}, \ \ \ \forall i \in M, j \in N
    \end{array}
$$

In [ ]:
model = Model();

In [ ]:
# Create y variables
@variable(model, y[1:m], Bin);
@variable(model, x[1:m, 1:n], Bin);

In [ ]:
# set objective
F = sum([f[j]*y[j] for j in 1:n])
C = sum([c[i, j]*x[i, j] for i in 1:m for j in 1:n])
                
@objective(model, Min, F + C);

In [ ]:
# Add constraints

# Each client is serve exactly once
@constraint(
    model,
    [i in 1:m],
    sum(x[i, j] for j in 1:n) == 1
)

# Fixed cost of opening facilities
@constraint(
    model,
    [i in 1:m, j in 1:n],
    x[i, j] <= y[j]
);

In [ ]:
# Set optimizer
set_optimizer(model, with_optimizer(GLPK.Optimizer))

In [ ]:
optimize!(model)
println("Optimal value: ", objective_value(model))

In [ ]:
# Get y and x solutions
ysol = value.(y);
println("Optimal solution y: ", value.(y))
xsol = value.(x);
println("Optimal solution x: ", value.(x))

In [ ]:
# relax all binary variables
for var in x
    is_binary(var) && unset_binary(var)
    set_lower_bound(var, 0.0)
    set_upper_bound(var, 1.0)
end

for var in y
    is_binary(var) && unset_binary(var)
    set_lower_bound(var, 0.0)
    set_upper_bound(var, 1.0)
end

# Solve the LP relaxation
optimize!(model)
lp_val = objective_value(model)
println("Optimal value of relaxed model: ", lp_val)

In [ ]:
# Get y and x solutions
lp_ysol = value.(y);
println("Optimal solution y: ", value.(y))
lp_xsol = value.(x);
println("Optimal solution x: ", value.(x))

In [ ]:
# set all variables to be binary
for var in x
    set_binary(var)
end

for var in y
    set_binary(var)
end

optimize!(model)
mip_val = objective_value(model)
println("Optimal value of integer model: ", mip_val)

In [ ]:
(mip_val - lp_val) / mip_val  # integrality gap

## Capacitated Facility location

* Each client $i$ has a demand $a_{i}$, and each facility has a capacity $q_{j}$

$$
    \begin{array}{cl}
        \min_{x, y} \ \ \ &
            \sum_{i, j} c_{i, j} x_{i, j} + 
            \sum_{j} f_{j} y_{j}\\
        s.t. &
            \sum_{j} x_{i, j} = 1, \ \ \ \forall i \in M\\
        &   \sum_{i} a_{i} x_{i, j} \leq q_{j} y_{j}, \ \ \ \forall j \in N\\
        &   x_{i, j}, y_{j} \in \{0, 1\}, \ \ \ \forall i \in M, j \in N
    \end{array}
$$

In [ ]:
n = 10  # number of facility locations
m = 30  # number of clients

# Draw costs
Random.seed!(0)
f = rand(1000:10000, n);
c = rand(0:1000, m, n);

# Clients' demands
a = rand(1:10, m);

# Capacities
q = rand(30:40, n);

In [ ]:
# Instantiate an empty model
model_cap = Model();

In [ ]:
# Create variables
y = @variable(model_cap, y[1:n], Bin);
x = @variable(model_cap, x[1:m, 1:n], Bin);

In [ ]:
# set objective
C = sum([c[i, j]*x[i, j] for i in 1:m for j in 1:n])  # demand serving cost
F = sum([f[j]*y[j] for j in 1:n])  # fixed cost

@objective(model_cap, Min, C + F);

In [ ]:
# Add constraints

# Each client is serve exactly once
ctr_ = @constraint(
    model_cap,                       # add constraints to model
    [i in 1:m],                      # there are `m` constraints, indexed by `i`
    sum(x[i, j] for j in 1:n) == 1   # the actual constraint
)

# Capacity constraints
ctr_capacity = @constraint(
    model_cap,
    [j in 1:n],
    sum(a[i] * x[i, j] for i in 1:m) <= q[j]*y[j]
);

# Tighten LP relaxation
ctr_opening = @constraint(
    model_cap,
    [i in 1:m, j in 1:n],
    x[i, j] <= y[j]
);

In [ ]:
# Set optimizer
set_optimizer(
    model_cap,
    with_optimizer(
        GLPK.Optimizer,
        msg_lev=3,    # verbosity level
        tm_lim=10000  # time limit, in ms
    )
)

In [ ]:
# Best solution found so far
println("Optimal value: ", objective_value(model_cap))

In [ ]:
# try with Cbc
set_optimizer(model_cap, with_optimizer(Cbc.Optimizer))
optimize!(model_cap)